In [1]:
import d2lzh as d2l
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

(corpus_indices, char_to_idx, idx_to_char, vocab_size)= d2l.load_data_jay_lyrics()

In [2]:
nd.one_hot(nd.array([0, 2]), vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

In [3]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]

X = nd.arange(10).reshape((2, 5))
inputs = to_onehot(X, vocab_size)
inputs

[
 [[1. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>, 
 [[0. 1. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>, 
 [[0. 0. 1. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>, 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>, 
 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]
 <NDArray 2x1027 @cpu(0)>]

In [4]:
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
print('will use', ctx)

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

will use gpu(0)


In [5]:
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx), )

In [6]:
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q, = params
    H, =state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [7]:
state  = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1027), (2, 256))

In [8]:
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state, 
                num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        (Y, state) = rnn(X, state, params)
        if t < len(prefix) - 1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [9]:
predict_rnn('周杰', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            ctx, idx_to_char, char_to_idx)

'周杰饿刚自送娇才烟实虽袭'

In [10]:
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [11]:
# 本函数已保存在d2lzh包中方便以后使用
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                          vocab_size, ctx, corpus_indices, idx_to_char,
                          char_to_idx, is_random_iter, num_epochs, num_steps,
                          lr, clipping_theta, batch_size, pred_period,
                          pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()

    for epoch in range(num_epochs):
        if not is_random_iter:  # 如使用相邻采样，在epoch开始时初始化隐藏状态
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter:  # 如使用随机采样，在每个小批量更新前初始化隐藏状态
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else:  # 否则需要使用detach函数从计算图分离隐藏状态
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                # outputs有num_steps个形状为(batch_size, vocab_size)的矩阵
                (outputs, state) = rnn(inputs, state, params)
                # 拼接之后形状为(num_steps * batch_size, vocab_size)
                outputs = nd.concat(*outputs, dim=0)
                # Y的形状是(batch_size, num_steps)，转置后再变成长度为
                # batch * num_steps 的向量，这样跟输出的行一一对应
                y = Y.T.reshape((-1,))
                # 使用交叉熵损失计算平均分类误差
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx)  # 裁剪梯度
            d2l.sgd(params, lr, 1)  # 因为误差已经取过均值，梯度不用再做平均
            l_sum += l.asscalar() * y.size
            n += y.size

        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                    prefix, pred_len, rnn, params, init_rnn_state,
                    num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [12]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [13]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 67.942836, time 0.63 sec
 - 分开 我想要你想你的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯
 - 不分开  我有你这想你的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让我
epoch 100, perplexity 10.105836, time 0.63 sec
 - 分开 一直在我心腔的怒火鸠 包地安老斑鸠 腿着安不多 除天一步四喝 为上入 一直两步三步四颗 连成线背著
 - 不分开吗 我有你的爱你在西元前 深埋在美索不达米亚平原 看十个世字刻 一直安我 说你说 语沉怎么 有一放空
epoch 150, perplexity 2.854726, time 0.66 sec
 - 分开 一只用双现 白一躺在抽边 它什么 让什么好气 辛欢苦 说弄堂 是属些三 语头放  我却睛好 我 没
 - 不分开扫 我有你爸 你打我妈 这样 看去好听 一指伦哭 在的完美 说让不知 你一定空 你却的梦 你有没  
epoch 200, perplexity 1.582395, time 0.63 sec
 - 分开 一只令 干步怎只系在四句 连成线掌著背默默许下心愿 看远方的星是否听的见 手牵手 一步两步三步四步
 - 不分开期把的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 在小村外的溪边河口默默等著我 娘
epoch 250, perplexity 1.305121, time 0.62 sec
 - 分开 出只忙 教在千武术在老 干什么 干什么 气被丹一起踢  原穿风霜千符 快果在双截  一悔在假动 一
 - 不分开扫 我不能再想 我不 我不 我不能 爱情走的太快就像龙卷风 不能承受我已无处可躲 我不要再想 我不要


In [14]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, False, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 56.640454, time 0.63 sec
 - 分开 我想要这爱 我不能再不 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我不能再想 我
 - 不分开 我有你这 你有我有 你有一直 如果我 别怪我有 你不我有 你是我有 你是我有 你是我的 快果我 别
epoch 100, perplexity 6.257481, time 0.62 sec
 - 分开 我想就这样牵在你的手不放开 爱可不可以简简单单没有演 看远方的星如果听的见 它一定实现 一色寄美 
 - 不分开一 就谁 回A 却后 弓箭 是谁的热旧 我不能再不 我不 我不 我不能再想 我不 我不 我不要再想你
epoch 150, perplexity 1.922529, time 0.66 sec
 - 分开 我静就这样活 唱着你 一直走 我想就这样牵着你的手 它一定实现 它的完美 在伤是一 还偷出空 你去
 - 不分开一 用谁黄上 我满后发 我都说你有力听我 就要说 说怪我 没有球酒 周来一碗热粥 配上几斤的牛肉 我
epoch 200, perplexity 1.272423, time 0.60 sec
 - 分开 问候的 旧颗我 一九四那 连漠线中的你边河口默默等著我 娘子依旧每日折一枝杨柳 你在那里 在小村外
 - 不分开一个的 什 我都你烦棒不 我这上没 泪我一定是我怎错 连成说危险不想 但那你人已经不是我 上海你在在
epoch 250, perplexity 1.155127, time 0.60 sec
 - 分开 问候的 旧酋我 你满了有 你杰伦 才离开没多久就开始 担心今天的你过得好不好 整个画面是你 想你想
 - 不分开一个的我 无子你不 我有多难熬  没有你在我有多难熬多烦恼  没有你烦 我有多烦恼  没有你烦我有多
